In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
import numpy as np
import time

In [2]:
df = pd.read_hdf('Global_Weather_Data/big_data.h5')
df.head()

,mean_temperature_7,mean_temperature_6,mean_temperature_5,mean_temperature_4,mean_temperature_3,mean_temperature_2,mean_temperature_1,day_of_year,altitude,latitude,...,wind_speed_0_2,wind_speed_0_3,wind_speed_0_4,wind_speed_0_5,wind_speed_0_6,wind_speed_0_7,wind_speed_0_8,wind_speed_0_9,wind_speed_0_10,wind_speed_0_11
3,299.906042,301.140417,299.718333,299.908125,300.634792,302.856458,302.795433,275,1148,29.558050,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
6,296.587500,296.475000,297.020000,296.770833,297.652500,297.247500,301.199792,276,2474,31.769039,...,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
7,298.190000,297.607500,298.117500,298.035417,298.867500,298.677500,300.139167,276,46,33.005859,...,6.0,6.0,6.0,6.0,3.0,3.0,3.0,3.0,3.0,3.0
8,298.190000,297.607500,298.117500,298.035417,298.867500,298.677500,300.197708,276,33,32.815559,...,6.0,6.0,6.0,6.0,3.0,3.0,3.0,3.0,3.0,3.0
9,298.917917,299.906042,301.140417,299.718333,299.908125,300.634792,302.856458,276,1148,29.558050,...,5.0,3.0,3.0,7.0,7.0,7.0,6.0,5.0,8.0,6.0


In [3]:
try:
    n_pred = int(df.columns[0][-2:])
except:
    n_pred = int(df.columns[0][-1])

In [4]:
df_mean = df.mean()
df_std = df.std()
df = (df-df.mean())/df.std()
df_train, df_test = train_test_split(df,test_size=0.1,random_state=1)
X_train = df_train.iloc[:,n_pred:].reset_index(drop=True)
# X_train_mean = X_train.mean()
# X_train_std = X_train.std()
# X_train = (X_train-X_train.mean())/X_train.std()

y_train = df_train.iloc[:,:n_pred].reset_index(drop=True)

X_test = df_test.iloc[:,n_pred:].reset_index(drop=True)
# X_test_mean = X_test.mean()
# X_test_std = X_test.std()
# X_test = (X_test-X_test.mean())/X_test.std()

y_test = df_test.iloc[:,:n_pred].reset_index(drop=True)

In [5]:
# grid = {'n_estimators':np.arange(10,301,10), 'max_depth':np.arange(1,30,1),'learning_rate':[1e-3,1e-2,1e-1]}
# grid = {'n_estimators':np.arange(10,201,10), 'max_depth':np.arange(1,20,1),'learning_rate':[1e-1,1e-2,1e-3]}
# grid = {'n_estimators':np.arange(500,1501,100), 'max_depth':np.arange(1,10,2),'learning_rate':np.arange(0.05,0.16,0.02)}
# grid = {'n_estimators':[500], 'max_depth':np.arange(6,9,1),'learning_rate':[0.1]}
# grid = {'n_estimators':np.arange(400,1001,100), 'max_depth':np.arange(5,11,1),'learning_rate':[0.1]}
grid = {'n_estimators':[500], 'max_depth':[7],'learning_rate':[0.1]}
gbt = GradientBoostingRegressor()
gbtCV = GridSearchCV(gbt,param_grid=grid,return_train_score=True,n_jobs=-1,verbose=1)

In [6]:
for i in range(n_pred,0,-1):
    start = time.time()
    idx = n_pred-i
    gbtCV.fit(X_train,y_train.iloc[:,idx])
    score = gbtCV.best_estimator_.score(X_test,y_test.iloc[:,idx])
    print('Forecasting mean temperature',i,'days ahead with 24 hours of history:')
    print('\tBest depth:\t\t', gbtCV.best_params_['max_depth'], '\n\tBest # estimators:\t', gbtCV.best_params_['n_estimators'],
          '\n\tBest learning rate:\t', gbtCV.best_params_['learning_rate'], '\n\tTraining R^2\t =',gbtCV.best_score_)
    print('\n\tValidation R^2\t=',score)
    
    baseline_score = r2_score(y_test.iloc[:,idx],X_test.iloc[:,1:25].mean(axis=1))
    print('\tBaseline\t=',baseline_score)
    if score > baseline_score:
        print('\tGood!')
    else:
        print('\tBad!')
    end = time.time()
    print('\nTotal minutes =',(end-start)/60)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Forecasting mean temperature 7 days ahead with 24 hours of history:
	Best depth:		 7 
	Best # estimators:	 500 
	Best learning rate:	 0.1 
	Training R^2	 = 0.9047899760715445

	Validation R^2	= 0.909113257187651
	Baseline	= 0.7737052927346195
	Good!

Total minutes = 4.704326363404592
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Forecasting mean temperature 6 days ahead with 24 hours of history:
	Best depth:		 7 
	Best # estimators:	 500 
	Best learning rate:	 0.1 
	Training R^2	 = 0.9036116601763204

	Validation R^2	= 0.9053532396546878
	Baseline	= 0.7859239331265391
	Good!

Total minutes = 4.672746455669403
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Forecasting mean temperature 5 days ahead with 24 hours of history:
	Best depth:		 7 
	Best # estimators:	 500 
	Best learning rate:	 0.1 
	Training R^2	 = 0.9058322810916886

	Validation R^2	= 0.9008947295236407
	Baseline	= 0.7897306522208862
	Good!



In [7]:
features = pd.DataFrame()
features['names'] = gbtCV.best_estimator_.feature_names_in_
features['importance'] = gbtCV.best_estimator_.feature_importances_
features.sort_values(by='importance',ascending=False)

,names,importance
21,temperature_0_6,0.375717
24,temperature_0_9,0.187460
22,temperature_0_7,0.138964
26,temperature_0_11,0.063109
11,temperature_0_20,0.052079
...,...,...
108,wind_speed_0_21,0.000079
109,wind_speed_0_22,0.000076
112,wind_speed_0_1,0.000070
113,wind_speed_0_2,0.000063


In [8]:
gbtCV.best_estimator_.predict(X_test.iloc[0:1,:])

array([-0.58535782])

In [9]:
# df.mean()

In [10]:
# df.std()

In [11]:
# y_test.iloc[0:1,:]

In [12]:
fun = features.sort_values(by='importance',ascending=False).reset_index(drop=True)
for i in range(features.shape[0]):
    print(fun.names[i],fun.importance[i])

temperature_0_6 0.3757167138327417
temperature_0_9 0.18745974114551128
temperature_0_7 0.1389642959166241
temperature_0_11 0.06310946374554989
temperature_0_20 0.05207879630518095
temperature_0_19 0.047381026698099876
temperature_0_18 0.026593502820035746
temperature_0_10 0.02461890181949455
temperature_0_21 0.014300333413085542
temperature_0_17 0.010196860353466735
temperature_0_5 0.00604116205807651
day_of_year 0.0044951383686307935
temperature_0_1 0.003946556829763871
temperature_0_22 0.003579987576934152
temperature_0_2 0.003487622954040569
temperature_0_0 0.0018616773487047896
temperature_0_4 0.0018088648680206123
temperature_0_3 0.0016280756810365977
temperature_0_23 0.0014722839821542262
temperature_0_8 0.0014119661479754097
wind_direction_0_11 0.0007954081726230842
wind_direction_0_15 0.0007007170938318636
wind_direction_0_12 0.0006953697840256901
humidity_0_13 0.0006300321114738351
humidity_0_18 0.0006218775693490205
latitude 0.0005607176338904164
temperature_0_12 0.0005507171